In [1]:
import numpy as np
import wandb
import matplotlib.pyplot as plt
import sys
import math
from keras.datasets import fashion_mnist,mnist
import sklearn
from sklearn.model_selection import train_test_split
from copy import deepcopy

2025-03-16 18:36:29.133676: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-16 18:36:29.144621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742130389.159024   13387 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742130389.163122   13387 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-16 18:36:29.178136: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [8]:
class Neural_Net:
    def __init__(self,input_shape,number_of_hidden_layers,hidden_neurons_per_layer,activation_function,output_shape,type_of_init,L2reg_const):
        self.input_shape = input_shape
        self.number_of_hidden_layers = number_of_hidden_layers
        self.hidden_neurons_per_layer = hidden_neurons_per_layer
        self.output_shape = output_shape
        self.activation_function = activation_function
        self.type_of_init = type_of_init
        self.L2reg_const = L2reg_const
        self.W, self.B = self.weight_init(type_of_init)

    def weight_init(self,type_of_init):
        W = []
        B = []
        
        if type_of_init == 'Xavier':    
            W.append(np.random.randn(self.input_shape, self.hidden_neurons_per_layer[0]) * np.sqrt(6 / (self.input_shape + self.hidden_neurons_per_layer[0])))
            for i in range(1, self.number_of_hidden_layers):
                W.append(np.random.randn(self.hidden_neurons_per_layer[i-1], self.hidden_neurons_per_layer[i]) *np.sqrt(6 / (self.hidden_neurons_per_layer[i-1] + self.hidden_neurons_per_layer[i])))
        
            W.append(np.random.randn(self.hidden_neurons_per_layer[-1], self.output_shape)*np.sqrt(6 / (self.hidden_neurons_per_layer[-1] + self.output_shape)))
        elif type_of_init == 'random':            
            W.append(np.random.rand(self.input_shape,self.hidden_neurons_per_layer[0])-0.5)
            for i in range(1,self.number_of_hidden_layers):
                W.append(np.random.rand(self.hidden_neurons_per_layer[i-1],self.hidden_neurons_per_layer[i])-0.5)
            W.append(np.random.rand(self.hidden_neurons_per_layer[-1],self.output_shape)-0.5)
            
        B.append(np.random.rand(1,self.hidden_neurons_per_layer[0])-0.5)
        for i in range(1,self.number_of_hidden_layers):
            B.append(np.random.rand(1,self.hidden_neurons_per_layer[i])-0.5)
        
        B.append(np.random.rand(1,self.output_shape)-0.5)
        return W,B

    def feed_forward(self,data,W,B):
        A = []         #pre-activation output list
        H = []         #post-ativation output list

        input_d = data
        for i in range(self.number_of_hidden_layers):
            linear_out = np.dot(input_d,W[i]) + B[i]
            activation_out = self.activation(linear_out)
            A.append(linear_out)
            H.append(activation_out)
            input_d = activation_out

        y_pred = self.softmax(np.dot(H[-1],W[-1]) + B[-1])
        return A,H,y_pred

    def activation(self,x):
        if self.activation_function == 'ReLU':
            return self.ReLU(x)
        elif self.activation_function == 'tanh':
            return self.tanh(x)
        elif self.activation_function == 'sigmoid':
            return self.sigmoid(x)
        elif self.activation_function == 'linear':
            return x

    def activation_derivative(self,x):
        if self.activation_function == 'ReLU':
            return self.ReLU_derivative(x)
        elif self.activation_function == 'tanh':
            return self.tanh_derivative(x)
        elif self.activation_function == 'sigmoid':
            return self.sigmoid_derivative(x)
        elif self.activation_function == 'linear':
            return 1

    def ReLU(self,x):
        return np.maximum(0,x)

    def ReLU_derivative(self,x):
        return np.where(x > 0, 1, 0)
    
    def sigmoid(self,x):
        # x = np.clip(x, -500, 500)
        return 1/(1+np.exp(-x))

    def sigmoid_derivative(self,x):
        return self.sigmoid(x)*(1-self.sigmoid(x))

    def tanh(self,x):
        return np.tanh(x)

    def tanh_derivative(self,x):
        return 1 - self.tanh(x)**2

    def softmax(self,x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        out = exp_x / np.sum(exp_x, axis=1, keepdims=True)
        return out

    def loss(self,label,pred):
        pred = np.clip(pred, 1e-15, 1 - 1e-15)
        loss = -np.sum(label*np.log(pred+1e-18),axis=1)
        return np.mean(loss)

    def accuracy(self,data,label):
        A,H,y_prob = self.feed_forward(data,self.W,self.B)
        acc = 0
        for i in range(len(data)):
            if np.argmax(label[i]) == np.argmax(y_prob[i]):
                acc += 1
        return acc/data.shape[0]
        
    def sgd_train(self,epochs,learning_rate,batch_size,train_data,train_label,test_data,test_label):
        num_batches = train_data.shape[0]//batch_size
        remaining_data = train_data.shape[0]%batch_size
        for ep in range(epochs):
            for b_id in range(num_batches+1):                
                batch_start = b_id * batch_size
                batch_end = min((b_id + 1) * batch_size, train_data.shape[0])
                batch_data = train_data[batch_start:batch_end]
                batch_label = train_label[batch_start:batch_end]

                A,H,y_prob = self.feed_forward(batch_data,self.W,self.B)
                
                dw,db = self.backpropagation(A,H,self.W,self.B,batch_data,y_prob,batch_label)
                for i in range(self.number_of_hidden_layers+1):
                    self.W[i] -= learning_rate * (dw[i])
                    self.B[i] -= learning_rate * db[i]
                
                if(b_id+1)%100 == 0:
                    _,_,test_y_prob = self.feed_forward(test_data,self.W,self.B)
                    test_loss = self.loss(test_label,test_y_prob)
                    train_acc = self.accuracy(batch_data,batch_label)
                    train_loss = self.loss(batch_label,y_prob)
                    test_acc = self.accuracy(test_data,test_label)
                    wandb.log({'train_loss': train_loss, 'train_acc': train_acc, 'val_loss': test_loss, 'val_acc': test_acc})
                    sys.stdout.write(f"\rEpoch {ep + 1}/{epochs} - Batch {b_id + 1}/{num_batches} - train-loss: {train_loss:.6f} train-acc: {train_acc:.6f} test-loss:{test_loss:.6f} test-acc : {test_acc:.6f} ")
                    sys.stdout.flush()
            print()

    def momentum_train(self,epochs,learning_rate,batch_size,train_data,train_label,test_data,test_label,beta=0.9):
        momentum_W = [np.zeros_like(w) for w in self.W]
        momentum_B = [np.zeros_like(b) for b in self.B]
        num_batches = train_data.shape[0]//batch_size
        remaining_data = train_data.shape[0]%batch_size
        for ep in range(epochs):
            for b_id in range(num_batches+1):                
                batch_start = b_id * batch_size
                batch_end = min((b_id + 1) * batch_size, train_data.shape[0])
                batch_data = train_data[batch_start:batch_end]
                batch_label = train_label[batch_start:batch_end]

                W_copy = deepcopy(self.W)
                B_copy = deepcopy(self.B)
                A,H,y_prob = self.feed_forward(batch_data,W_copy,B_copy)

                
                dw,db = self.backpropagation(A,H,W_copy,B_copy,batch_data,y_prob,batch_label)
                for i in range(self.number_of_hidden_layers+1):
                    momentum_W[i] = beta*momentum_W[i] + (1-beta)*(dw[i])
                    momentum_B[i] = beta*momentum_B[i] + (1-beta)*db[i]
                    self.W[i] -= learning_rate * momentum_W[i]
                    self.B[i] -= learning_rate * momentum_B[i]
                
                if(b_id+1)%100 == 0:
                    _,_,test_y_prob = self.feed_forward(test_data,self.W,self.B)
                    test_loss = self.loss(test_label,test_y_prob)
                    train_acc = self.accuracy(batch_data,batch_label)
                    train_loss = self.loss(batch_label,y_prob)
                    test_acc = self.accuracy(test_data,test_label)
                    wandb.log({'train_loss': train_loss, 'train_acc': train_acc, 'val_loss': test_loss, 'val_acc': test_acc})
                    sys.stdout.write(f"\rEpoch {ep + 1}/{epochs} - Batch {b_id + 1}/{num_batches} - train-loss: {train_loss:.6f} train-acc: {train_acc:.6f} test-loss:{test_loss:.6f} test-acc : {test_acc:.6f} ")
                    sys.stdout.flush()
            print()

    def nestrov_train(self,epochs,learning_rate,batch_size,train_data,train_label,test_data,test_label,beta=0.9):
        momentum_W = [np.zeros_like(w) for w in self.W]
        momentum_B = [np.zeros_like(b) for b in self.B]
        num_batches = train_data.shape[0]//batch_size
        remaining_data = train_data.shape[0]%batch_size
        for ep in range(epochs):
            for b_id in range(num_batches+1):                
                batch_start = b_id * batch_size
                batch_end = min((b_id + 1) * batch_size, train_data.shape[0])
                batch_data = train_data[batch_start:batch_end]
                batch_label = train_label[batch_start:batch_end]

                # we want grad(wt - beta*momentum), so feed forward through this value and get grads
                W_lookahead = [w - beta*v for w,v in zip(self.W,momentum_W)]
                B_lookahead = [b - beta*v for b,v in zip(self.B,momentum_B)]
                
                
                A,H,y_prob = self.feed_forward(batch_data,W_lookahead,B_lookahead)
                dw,db = self.backpropagation(A,H,W_lookahead,B_lookahead,batch_data,y_prob,batch_label)
                for i in range(self.number_of_hidden_layers+1):
                    # dw[i] += self.L2reg_const * self.W[i]
                    momentum_W[i] = beta*momentum_W[i] + (dw[i])
                    momentum_B[i] = beta*momentum_B[i] + db[i]
                    
                    self.W[i] -= learning_rate * momentum_W[i]
                    self.B[i] -= learning_rate * momentum_B[i]
                
                if(b_id+1)%100 == 0:
                    _,_,test_y_prob = self.feed_forward(test_data,self.W,self.B)
                    test_loss = self.loss(test_label,test_y_prob)
                    train_acc = self.accuracy(batch_data,batch_label)
                    train_loss = self.loss(batch_label,y_prob)
                    test_acc = self.accuracy(test_data,test_label)
                    wandb.log({'train_loss': train_loss, 'train_acc': train_acc, 'val_loss': test_loss, 'val_acc': test_acc})
                    sys.stdout.write(f"\rEpoch {ep + 1}/{epochs} - Batch {b_id + 1}/{num_batches} - train-loss: {train_loss:.6f} train-acc: {train_acc:.6f} test-loss:{test_loss:.6f} test-acc : {test_acc:.6f} ")
                    sys.stdout.flush()
            print()

    def RMSprop_train(self,epochs,learning_rate,batch_size,train_data,train_label,test_data,test_label,beta,eps):
        v_W = [np.zeros_like(w) for w in self.W]
        v_B = [np.zeros_like(b) for b in self.B]
        num_batches = train_data.shape[0]//batch_size
        remaining_data = train_data.shape[0]%batch_size
        for ep in range(epochs):
            for b_id in range(num_batches+1):                
                batch_start = b_id * batch_size
                batch_end = min((b_id + 1) * batch_size, train_data.shape[0])
                batch_data = train_data[batch_start:batch_end]
                batch_label = train_label[batch_start:batch_end]

                W_copy = deepcopy(self.W)
                B_copy = deepcopy(self.B)
                A,H,y_prob = self.feed_forward(batch_data,W_copy,B_copy)

                
                
                dw,db = self.backpropagation(A,H,W_copy,B_copy,batch_data,y_prob,batch_label)
                for i in range(self.number_of_hidden_layers+1):
                    # dw[i] += self.L2reg_const * self.W[i]
                    
                    v_W[i] = beta*v_W[i] + (1-beta)*(dw[i]**2)
                    v_B[i] = beta*v_B[i] + (1-beta)*(db[i]**2)

                    adaptive_lr_w = learning_rate/(np.sqrt(v_W[i]+eps))
                    adaptive_lr_b = learning_rate/(np.sqrt(v_B[i]+eps))
                    self.W[i] -= adaptive_lr_w * (dw[i])
                    self.B[i] -= adaptive_lr_b * db[i]
                
                if(b_id+1)%100 == 0:
                    _,_,test_y_prob = self.feed_forward(test_data,self.W,self.B)
                    test_loss = self.loss(test_label,test_y_prob)
                    train_acc = self.accuracy(batch_data,batch_label)
                    train_loss = self.loss(batch_label,y_prob)
                    test_acc = self.accuracy(test_data,test_label)
                    wandb.log({'train_loss': train_loss, 'train_acc': train_acc, 'val_loss': test_loss, 'val_acc': test_acc})
                    sys.stdout.write(f"\rEpoch {ep + 1}/{epochs} - Batch {b_id + 1}/{num_batches} - train-loss: {train_loss:.6f} train-acc: {train_acc:.6f} test-loss:{test_loss:.6f} test-acc : {test_acc:.6f} ")
                    sys.stdout.flush()
            print()   

    def adam_train(self,epochs,learning_rate,batch_size,train_data,train_label,test_data,test_label,beta1=0.9,beta2=0.999,eps=1e-8):
        momentum1_W = [np.zeros_like(w) for w in self.W]
        momentum1_B = [np.zeros_like(b) for b in self.B]
        momentum2_W = [np.zeros_like(w) for w in self.W]
        momentum2_B = [np.zeros_like(b) for b in self.B]
        t = 0
        num_batches = train_data.shape[0]//batch_size
        remaining_data = train_data.shape[0]%batch_size
        for ep in range(epochs):
            for b_id in range(num_batches+1):
                t += 1
                batch_start = b_id * batch_size
                batch_end = min((b_id + 1) * batch_size, train_data.shape[0])
                batch_data = train_data[batch_start:batch_end]
                batch_label = train_label[batch_start:batch_end]

                W_copy = deepcopy(self.W)
                B_copy = deepcopy(self.B)
                A,H,y_prob = self.feed_forward(batch_data,W_copy,B_copy)
    
                
                dw,db = self.backpropagation(A,H,W_copy,B_copy,batch_data,y_prob,batch_label)
                for i in range(self.number_of_hidden_layers+1):
                    momentum1_W[i] = beta1*momentum1_W[i] + (1-beta1)*(dw[i])
                    momentum1_B[i] = beta1*momentum1_B[i] + (1-beta1)*(db[i])
    
                    momentum2_W[i] = beta2*momentum2_W[i] + (1-beta2)*(dw[i]**2)
                    momentum2_B[i] = beta2*momentum2_B[i] + (1-beta2)*(db[i]**2)
    
                    momentum1_W_hat = momentum1_W[i]/(1-(beta1**t))
                    momentum1_B_hat = momentum1_B[i]/(1-(beta1**t))
    
                    momentum2_W_hat = momentum2_W[i]/(1-(beta2**t))
                    momentum2_B_hat = momentum2_B[i]/(1-(beta2**t))
    
                    adaptive_lr_W = learning_rate/(np.sqrt(momentum2_W_hat) + eps)
                    adaptive_lr_B = learning_rate/(np.sqrt(momentum2_B_hat) + eps)
    
                    self.W[i] -= adaptive_lr_W * (momentum1_W_hat)
                    self.B[i] -= adaptive_lr_B * momentum1_B_hat
                    
                
                if(b_id+1)%100 == 0:
                    _,_,test_y_prob = self.feed_forward(test_data,self.W,self.B)
                    test_loss = self.loss(test_label,test_y_prob)
                    train_acc = self.accuracy(batch_data,batch_label)
                    train_loss = self.loss(batch_label,y_prob)
                    test_acc = self.accuracy(test_data,test_label)
                    # wandb.log({'train_loss': train_loss, 'train_acc': train_acc, 'val_loss': test_loss, 'val_acc': test_acc})
                    sys.stdout.write(f"\rEpoch {ep + 1}/{epochs} - Batch {b_id + 1}/{num_batches} - train-loss: {train_loss:.6f} train-acc: {train_acc:.6f} test-loss:{test_loss:.6f} test-acc : {test_acc:.6f} ")
                    sys.stdout.flush()
            print()

    def nadam_train(self,epochs,learning_rate,batch_size,train_data,train_label,test_data,test_label,beta1=0.9,beta2=0.999,eps=1e-8):
        momentum1_W = [np.zeros_like(w) for w in self.W]
        momentum1_B = [np.zeros_like(b) for b in self.B]
        momentum2_W = [np.zeros_like(w) for w in self.W]
        momentum2_B = [np.zeros_like(b) for b in self.B]
        t = 0
        num_batches = train_data.shape[0]//batch_size
        remaining_data = train_data.shape[0]%batch_size
        for ep in range(epochs):
            for b_id in range(num_batches+1):
                t += 1
                batch_start = b_id * batch_size
                batch_end = min((b_id + 1) * batch_size, train_data.shape[0])
                batch_data = train_data[batch_start:batch_end]
                batch_label = train_label[batch_start:batch_end]
                W_copy = deepcopy(self.W)
                B_copy = deepcopy(self.B)
                A,H,y_prob = self.feed_forward(batch_data,W_copy,B_copy)
                
                dw,db = self.backpropagation(A,H,W_copy,B_copy,batch_data,y_prob,batch_label)
                for i in range(self.number_of_hidden_layers+1):
                    # dw[i] += self.L2reg_const * self.W[i]
                    momentum1_W[i] = beta1*momentum1_W[i] + (1-beta1)*(dw[i])
                    momentum1_B[i] = beta1*momentum1_B[i] + (1-beta1)*(db[i])
    
                    momentum2_W[i] = beta2*momentum2_W[i] + (1-beta2)*(dw[i]**2)
                    momentum2_B[i] = beta2*momentum2_B[i] + (1-beta2)*(db[i]**2)
    
                    momentum1_W_hat = momentum1_W[i]/(1-(beta1**t))
                    momentum1_B_hat = momentum1_B[i]/(1-(beta1**t))
    
                    momentum2_W_hat = momentum2_W[i]/(1-(beta2**t))
                    momentum2_B_hat = momentum2_B[i]/(1-(beta2**t))
    
                    m_nestrov_W = beta1 * momentum1_W_hat + ((1 - beta1) * dw[i])/(1-beta1**t)
                    m_nestrov_B = beta1 * momentum1_B_hat + ((1 - beta1) * db[i])/(1-beta1**t)
    
                    adaptive_lr_W = learning_rate / (np.sqrt(momentum2_W_hat) + eps)
                    adaptive_lr_B = learning_rate / (np.sqrt(momentum2_B_hat) + eps)
    
                    self.W[i] -= adaptive_lr_W * m_nestrov_W
                    self.B[i] -= adaptive_lr_B * m_nestrov_B
                    
                
                if(b_id+1)%100 == 0:
                    _,_,test_y_prob = self.feed_forward(test_data,self.W,self.B)
                    test_loss = self.loss(test_label,test_y_prob)
                    train_acc = self.accuracy(batch_data,batch_label)
                    train_loss = self.loss(batch_label,y_prob)
                    test_acc = self.accuracy(test_data,test_label)
                    wandb.log({'train_loss': train_loss, 'train_acc': train_acc, 'val_loss': test_loss, 'val_acc': test_acc})
                    sys.stdout.write(f"\rEpoch {ep + 1}/{epochs} - Batch {b_id + 1}/{num_batches} - train-loss: {train_loss:.6f} train-acc: {train_acc:.6f} test-loss:{test_loss:.6f} test-acc : {test_acc:.6f} ")
                    sys.stdout.flush()
            print()
        
    def train(self,optimizer,epochs,learning_rate,batch_size,train_data,train_label,test_data,test_label):
        if optimizer == 'sgd':
            self.sgd_train(epochs,learning_rate,batch_size,train_data,train_label,test_data,test_label)
        elif optimizer == 'momentum':
            self.momentum_train(epochs,learning_rate,batch_size,train_data,train_label,test_data,test_label,0.9)
        elif optimizer == 'nestrov':
            self.nestrov_train(epochs,learning_rate,batch_size,train_data,train_label,test_data,test_label,0.9)
        elif optimizer == 'RMSprop':
            self.RMSprop_train(epochs,learning_rate,batch_size,train_data,train_label,test_data,test_label,0.8,1e-8)
        elif optimizer == 'adam':
            self.adam_train(epochs,learning_rate,batch_size,train_data,train_label,test_data,test_label,0.9,0.999,1e-8)
        elif optimizer == 'nadam':
            self.nadam_train(epochs,learning_rate,batch_size,train_data,train_label,test_data,test_label,0.9,0.999,1e-8)
            

    def backpropagation(self,A,H,W,B,batch_data,y_prob,batch_label):
        dW = []
        db = []
        
        # Calculate initial error (cross-entropy gradient for softmax)
        error = y_prob - batch_label  
        # Backpropagate through output layer (softmax)
        dW.insert(0, np.dot(H[-1].T, error)+self.L2reg_const * W[-1])
        db.insert(0, np.sum(error,axis=0,keepdims=True))
        
        # Backpropagate through hidden layers
        delta = error
        for i in range(self.number_of_hidden_layers - 1, 0, -1):
            delta = np.dot(delta, W[i+1].T) * self.activation_derivative(A[i])
            dW.insert(0, np.dot(H[i-1].T, delta)+ self.L2reg_const * W[i])
            db.insert(0, np.sum(delta, axis=0,keepdims=True))
        
        # Backpropagate through input layer
        delta = np.dot(delta, W[1].T) * self.activation_derivative(A[0])
        dW.insert(0, np.dot(batch_data.T, delta)+ self.L2reg_const * W[0])
        db.insert(0, np.sum(delta, axis=0,keepdims=True))
        # print(dW)
        # input()
        return dW, db



    

In [5]:
def train():
    var1 = wandb.init(project='dl-assignment1-experiment')
    config = var1.config

    input_size = train_data.shape[1]
    num_hidden_layers = config.num_of_hidden_layers

    num_neurons_each_layer = [ config.hidden_layer_size ] * num_hidden_layers 
    activation_function = config.activation_function
    type_of_init = config.weight_initialization
    L2reg_const = config.weight_decay

    run_name = f"hl_{config.num_of_hidden_layers}_bs_{config.batch_size}_ac_{config.activation_function}"
    
    wandb.run.name = run_name
    wandb.run.save()

    print(f"Starting training with run name: {run_name}")
    nn = Neural_Net(
        number_of_hidden_layers=num_hidden_layers,
        hidden_neurons_per_layer=num_neurons_each_layer,
        activation_function=activation_function,
        input_shape=input_size,
        type_of_init=type_of_init,
        L2reg_const=L2reg_const,
        output_shape = 10,
    )
    
    nn.train(
        optimizer=config.optimizer,
        epochs=config.epochs,
        learning_rate=config.learning_rate,
        train_data=train_data,  # Make sure to define train_data and labels
        train_label=train_label,
        test_data=val_data,
        test_label=val_label,
        batch_size=config.batch_size
    )

In [6]:
sweep_config = {
  'name': 'fashion-mnist-exp(random-select)-2.10',
  'method': 'bayes',
  'metric': {'goal': 'maximize', 'name': 'val_acc'},
  'parameters': {
        'num_of_hidden_layers':{'values':[3,4,5]},
        'hidden_layer_size': {'values': [32, 64, 128]},
      'activation_function': {'values': ['sigmoid', 'tanh', 'ReLU']},
      'batch_size': {'values': [16, 32, 64]},
      'epochs': {'values': [5, 10]},
      'learning_rate': {'values': [1e-3,1e-4]},
      'optimizer': {'values': ['sgd', 'momentum', 'nesterov', 'RMSprop', 'adam', 'nadam']},
      'weight_initialization': {'values': ['random', 'Xavier']},
      'weight_decay': {'values': [0, 0.0005,0.5]},
  }
}

In [7]:
sweep_id = wandb.sweep(sweep_config,project='dl-assignment1-experiment')
wandb.agent(sweep_id,train,count=50)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: hfa2u67m
Sweep URL: https://wandb.ai/cs24s031-indian-institute-of-technology-madras/dl-assignment1-experiment/sweeps/hfa2u67m


wandb: Agent Starting Run: 5x2w277x with config:
wandb: 	activation_function: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	hidden_layer_size: 128
wandb: 	learning_rate: 0.001
wandb: 	num_of_hidden_layers: 5
wandb: 	optimizer: nesterov
wandb: 	weight_decay: 0.5
wandb: 	weight_initialization: Xavier
wandb: Currently logged in as: cs24s031 (cs24s031-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Starting training with run name: hl_5_bs_16_ac_sigmoid


...

Starting training with run name: hl_4_bs_64_ac_ReLU
Epoch 1/10 - Batch 800/843 - train-loss: 0.443855 train-acc: 0.859375 test-loss:0.429825 test-acc : 0.844500 
Epoch 2/10 - Batch 800/843 - train-loss: 0.461144 train-acc: 0.828125 test-loss:0.428406 test-acc : 0.847333 
Epoch 3/10 - Batch 800/843 - train-loss: 0.463035 train-acc: 0.843750 test-loss:0.427606 test-acc : 0.845500 
Epoch 4/10 - Batch 800/843 - train-loss: 0.460699 train-acc: 0.843750 test-loss:0.422786 test-acc : 0.847667 
Epoch 5/10 - Batch 800/843 - train-loss: 0.458521 train-acc: 0.828125 test-loss:0.419889 test-acc : 0.850500 
Epoch 6/10 - Batch 800/843 - train-loss: 0.460978 train-acc: 0.828125 test-loss:0.416605 test-acc : 0.850500 
Epoch 7/10 - Batch 800/843 - train-loss: 0.444373 train-acc: 0.843750 test-loss:0.413389 test-acc : 0.853333 
Epoch 8/10 - Batch 800/843 - train-loss: 0.447468 train-acc: 0.859375 test-loss:0.409889 test-acc : 0.853333 
Epoch 9/10 - Batch 800/843 - train-loss: 0.438167 train-acc: 0.82812

train_acc,▁▅▃▃▆▃▆▃▅▂▂▄▃█▃▇▅▆▃▃▃▃▆▂▅▂▇▃▅▆▃▄▇▂▃▂▃▅▃▂
train_loss,█▆▅▆█▁▅▃▂▆▅▅▁▅▃▆█▅▅▁▅▇▅▅▅▂▆▂▃▆▅▃▂▆▃▇▁▂▃▁
val_acc,▁▅▆▇▇█▇▇▇█▇▇▇██▇▇▆▇▇▆▆▇▇▇▇█▇▇█▇▇██▇███▇█
val_loss,█▄▄▃▂▁▂▂▁▃▂▁▃▂▃▂▁▃▁▂▂▂▁▂▁▂▁▁▂▂▂▁▁▂▂▁▁▂▂▁
train_acc,0.82812
train_loss,0.43765
val_acc,0.8555
val_loss,0.40635


wandb: Agent Starting Run: no10hyno with config:
wandb: 	activation_function: ReLU
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 128
wandb: 	learning_rate: 0.0001
wandb: 	num_of_hidden_layers: 5
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0
wandb: 	weight_initialization: Xavier


Starting training with run name: hl_5_bs_16_ac_ReLU
Epoch 1/10 - Batch 3300/3375 - train-loss: 0.733194 train-acc: 0.812500 test-loss:0.533488 test-acc : 0.810667 
Epoch 2/10 - Batch 3300/3375 - train-loss: 0.720832 train-acc: 0.750000 test-loss:0.465561 test-acc : 0.832667 
Epoch 3/10 - Batch 3300/3375 - train-loss: 0.614617 train-acc: 0.812500 test-loss:0.435430 test-acc : 0.844667 
Epoch 4/10 - Batch 3300/3375 - train-loss: 0.583512 train-acc: 0.812500 test-loss:0.420452 test-acc : 0.850333 
Epoch 5/10 - Batch 3300/3375 - train-loss: 0.566065 train-acc: 0.812500 test-loss:0.406387 test-acc : 0.856167 
Epoch 6/10 - Batch 3300/3375 - train-loss: 0.541091 train-acc: 0.812500 test-loss:0.398145 test-acc : 0.855000 
Epoch 7/10 - Batch 3300/3375 - train-loss: 0.535813 train-acc: 0.812500 test-loss:0.389518 test-acc : 0.857000 
Epoch 8/10 - Batch 3300/3375 - train-loss: 0.549111 train-acc: 0.812500 test-loss:0.383347 test-acc : 0.859833 
Epoch 9/10 - Batch 3300/3375 - train-loss: 0.537301 

train_acc,▆▅▃▆█▅▆▆▆▃▆▃▁▆█▆▆█▆██▅▅▆▅▆███▃▆▆▃█▁▆▅▅▆█
train_loss,▆▆▄▃▆▄█▂▅▂▃▄▃▃▃▂▃▄▂▂▇▂▂▂▁▇▂▃▃▃▁▂▃▄▂▁▂▂▂▃
val_acc,▁▆▆▆▆▇▇▇▇▇▇█▇▇▇████▇█▇██████████████████
val_loss,██▅▄▄▃▄▃▄▃▃▄▃▂▃▂▂▂▃▂▂▃▃▂▂▃▂▂▂▂▁▁▁▁▂▁▂▁▂▁
train_acc,0.8125
train_loss,0.52439
val_acc,0.86317
val_loss,0.37663


In [11]:
wandb.finish()

In [12]:
nn = Neural_Net(784,4,[32,32,32,32],'ReLU',10,'Xavier',0.005)
nn.train('adam',10,0.001,32,train_data,train_label,val_data,val_label)

Epoch 1/10 - Batch 1600/1687 - train-loss: 0.489906 train-acc: 0.781250 test-loss:0.484841 test-acc : 0.820000 
Epoch 2/10 - Batch 400/1687 - train-loss: 0.436420 train-acc: 0.843750 test-loss:0.430519 test-acc : 0.839500 

KeyboardInterrupt: 

In [10]:
nn